In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from vivarium import InteractiveContext, Artifact

from datetime import datetime, timedelta
from pathlib import Path
import itertools
import matplotlib.pyplot as plt
import ipywidgets
import pandas as pd, numpy as np
pd.set_option('display.max_rows', 60)

import numpy as np
import researchpy as rp
import statsmodels.api as sm
import statsmodels.formula.api as smf
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
import scipy.stats

from db_queries import get_outputs as go
from db_queries import get_ids
from get_draws.api import get_draws

## Load in Data and First Time Step

In [2]:
path = Path('/ihme/homes/abie/projects/2023/vivarium_nih_us_cvd/src/vivarium_nih_us_cvd/model_specifications/nih_us_cvd.yaml')
sim = InteractiveContext(path, setup=False)

In [3]:
sim.configuration.update({
                          'population':
                              {'population_size': 10_000,
                               'age_start': 40,
                              },
                          'time':
                              {'start':
                                  {'year': 2021
                                  }
                              }
                          }
                        )
sim.setup()

2023-10-30 14:05:01.050 | INFO     | simulation_1-artifact_manager:67 - Running simulation from artifact located at /mnt/team/simulation_science/pub/models/vivarium_nih_us_cvd/artifacts/51-locations/v3-20231019/alabama.hdf.
2023-10-30 14:05:01.055 | INFO     | simulation_1-artifact_manager:68 - Artifact base filter terms are ['draw == 0'].
2023-10-30 14:05:01.058 | INFO     | simulation_1-artifact_manager:69 - Artifact additional filter terms are None.


In [4]:
pop0 = sim.get_population()
pop0.head()

,tracked,age,alive,sex,entrance_time,location,exit_time,years_of_life_lost,cause_of_death,previous_ischemic_stroke,...,ldlc_medication_adherence,sbp_multiplier,ldlc_therapeutic_inertia_constant_component,polypill,ldlc_multiplier,sbp_therapeutic_inertia_constant_component,lifestyle_adherence,visit_type,scheduled_date,last_fpg_test_date
0,True,62.325405,alive,Female,2020-12-04,Alabama,NaT,0.0,not_dead,,...,cat3,1.00,0.095895,cat2,1.3620,-0.050506,False,none,NaT,NaT
1,True,56.583394,alive,Male,2020-12-04,Alabama,NaT,0.0,not_dead,,...,cat3,1.00,0.528809,cat2,1.5125,0.901321,False,none,2021-03-21 23:06:47.554560,NaT
2,True,63.072592,alive,Female,2020-12-04,Alabama,NaT,0.0,not_dead,,...,cat3,1.00,0.558211,cat2,1.0000,-0.448101,True,none,NaT,NaT
3,True,50.472918,alive,Male,2020-12-04,Alabama,NaT,0.0,not_dead,,...,cat1,1.12,-0.027969,cat2,1.0000,0.133548,True,none,NaT,NaT
4,True,49.552787,alive,Female,2020-12-04,Alabama,NaT,0.0,not_dead,,...,cat3,1.00,-0.353543,cat2,1.3620,-0.216679,True,none,NaT,NaT


In [5]:
pop0.columns

Index(['tracked', 'age', 'alive', 'sex', 'entrance_time', 'location',
       'exit_time', 'years_of_life_lost', 'cause_of_death',
       'previous_ischemic_stroke',
       'previous_ischemic_heart_disease_and_heart_failure',
       'high_ldl_cholesterol_propensity',
       'high_body_mass_index_in_adults_propensity',
       'high_systolic_blood_pressure_propensity',
       'high_fasting_plasma_glucose_propensity', 'ischemic_stroke',
       'ischemic_heart_disease_and_heart_failure',
       'ensemble_propensity_risk_factor.high_ldl_cholesterol',
       'ensemble_propensity_risk_factor.high_systolic_blood_pressure',
       'ensemble_propensity_risk_factor.high_body_mass_index_in_adults',
       'ensemble_propensity_risk_factor.high_fasting_plasma_glucose',
       'sbp_medication_adherence_propensity',
       'ldlc_medication_adherence_propensity', 'outreach_propensity',
       'polypill_propensity', 'lifestyle_propensity',
       'susceptible_to_ischemic_stroke_event_count',
       'susc

In [6]:
sim.list_values()

['metrics',
 'cause_specific_mortality_rate',
 'acute_ischemic_stroke.incidence_rate',
 'acute_ischemic_stroke.incidence_rate.paf',
 'acute_ischemic_stroke.dwell_time',
 'acute_ischemic_stroke.disability_weight',
 'disability_weight',
 'acute_ischemic_stroke.excess_mortality_rate',
 'acute_ischemic_stroke.excess_mortality_rate.paf',
 'mortality_rate',
 'chronic_ischemic_stroke.dwell_time',
 'chronic_ischemic_stroke.disability_weight',
 'chronic_ischemic_stroke.excess_mortality_rate',
 'chronic_ischemic_stroke.excess_mortality_rate.paf',
 'chronic_ischemic_stroke_to_acute_ischemic_stroke.transition_rate',
 'chronic_ischemic_stroke_to_acute_ischemic_stroke.transition_rate.paf',
 'acute_myocardial_infarction.incidence_rate',
 'heart_failure_from_ischemic_heart_disease.incidence_rate',
 'heart_failure_residual.incidence_rate',
 'acute_myocardial_infarction.incidence_rate.paf',
 'heart_failure_from_ischemic_heart_disease.incidence_rate.paf',
 'heart_failure_residual.incidence_rate.paf',
 's

In [7]:
data1 = pd.concat([pop0,
                   sim.get_value('high_systolic_blood_pressure.exposure')(pop0.index).rename('high_sbp'),
                   sim.get_value('high_systolic_blood_pressure.raw_exposure')(pop0.index).rename('high_sbp_raw'),
                  ], axis=1)
data1.head()

,tracked,age,alive,sex,entrance_time,location,exit_time,years_of_life_lost,cause_of_death,previous_ischemic_stroke,...,ldlc_therapeutic_inertia_constant_component,polypill,ldlc_multiplier,sbp_therapeutic_inertia_constant_component,lifestyle_adherence,visit_type,scheduled_date,last_fpg_test_date,high_sbp,high_sbp_raw
0,True,62.325405,alive,Female,2020-12-04,Alabama,NaT,0.0,not_dead,,...,0.095895,cat2,1.3620,-0.050506,False,none,NaT,NaT,147.192049,147.192049
1,True,56.583394,alive,Male,2020-12-04,Alabama,NaT,0.0,not_dead,,...,0.528809,cat2,1.5125,0.901321,False,none,2021-03-21 23:06:47.554560,NaT,145.689753,145.689753
2,True,63.072592,alive,Female,2020-12-04,Alabama,NaT,0.0,not_dead,,...,0.558211,cat2,1.0000,-0.448101,True,none,NaT,NaT,94.942811,94.942811
3,True,50.472918,alive,Male,2020-12-04,Alabama,NaT,0.0,not_dead,,...,-0.027969,cat2,1.0000,0.133548,True,none,NaT,NaT,148.753688,165.153688
4,True,49.552787,alive,Female,2020-12-04,Alabama,NaT,0.0,not_dead,,...,-0.353543,cat2,1.3620,-0.216679,True,none,NaT,NaT,139.227528,139.227528


In [8]:
data1 = data1.loc[(data1.age > 25) & (data1.alive == 'alive')]
data1['age_start'] = ((data1.age/5).astype(int) * 5).astype(float)
#data1.loc[data1.sbp_medication != 'no_treatment']

## Running time forward 

In [16]:
discontinue_risk = .3/12  # probability of discontinuing treatment during a single time step
# to be applied for the first year of treatment

In [17]:
## This is the function that reset people on meds back to no treatment. 
## I would expect that this is where you will make edits 

global_medication_start = None

def reset_sbp_medication(pop_t, current_timestep):
    global global_medication_start
    if global_medication_start is None: # initialize medication start data
        global_medication_start = pd.Series(-1_000, index=pop_t.index)
        
    ##Call the component with treatment 
    treatment = sim.get_component('treatment')
    ##Get a population view 
    data = treatment.population_view.get(pop_t.index)
    
    # record timestep that this changed
    new_med_rows = (data['sbp_medication'] != 'no_treatment') & (global_medication_start < 0)
    global_medication_start[new_med_rows] = current_timestep
    
    
    ##Actual edit the data 
    ##More specifically this is where we will make edits to have the first year reset 
    rows_to_consider = ((data['sbp_medication'] != 'no_treatment')
                        & (global_medication_start >= current_timestep - 12))
    
    indices_to_change = rows_to_consider & (np.random.uniform(size=len(rows_to_consider)) <= discontinue_risk)
    data.loc[indices_to_change, 'sbp_medication'] = 'no_treatment'
    global_medication_start[indices_to_change] = -1_000  # this negative value means "not on treatment"
    
    ##Update the data in the population view with the new dataframe that was edited 
    treatment.population_view.update(data)
    return 

In [ ]:
%%time

## I would reset this to 1 time step or 5 time steps to test it 

sim_data = pd.DataFrame()
for step in list(range(0,36)):
    sim.step()
    pop_t = sim.get_population()
    reset_sbp_medication(pop_t, step) 
    data_t = pd.concat([pop_t, 
                   sim.get_value('high_systolic_blood_pressure.exposure')(pop0.index).rename('high_sbp'),
                   sim.get_value('high_systolic_blood_pressure.raw_exposure')(pop0.index).rename('high_sbp_raw'),
                       ],axis=1)
    data_t['step'] = step
    data_t_small = data_t[['sex','age','alive','high_sbp','high_sbp_raw','sbp_medication','step']]
    print(step)
    sim_data = pd.concat([sim_data, data_t_small])

0
1
2
3


In [ ]:
## Save the data - I find this helpful so that if things happen overnight (internet outage, jupyter crash) I still have a copy 

sim_data.to_csv('data/med_data_250steps_10_27_23.csv')

# Read in Data and Analysis 

In [ ]:
data = pd.read_csv('data/med_data_250steps_10_27_23.csv')

In [ ]:
data1 = data.loc[(data.age > 25) & (data.alive == 'alive')]
data1['age_start'] = ((data1.age/5).astype(int) * 5).astype(float)
data1['need_meds'] = np.where(data1.high_sbp_raw > 130, 1, 0)

In [ ]:
data_need_med = data1.loc[data1.need_meds == 1]
data_need_med.head()

In [ ]:
grouped = data_need_med.groupby(['sex','age_start','step','sbp_medication']).size()
errors = pd.DataFrame(grouped).reset_index()
errors['error'] = np.sqrt(errors[0])
#errors
percentages = grouped / data_need_med.groupby(['sex','age_start','step']).size() * 100
percentages= pd.DataFrame(percentages)
percentages = percentages.reset_index()
percentages = percentages.merge(errors[['sex','age_start','step','sbp_medication','error']], on = ['sex','age_start','step','sbp_medication'])
percentages['final_error'] = percentages[0].std() / percentages['error']
percentages

In [ ]:
for sex in percentages.sex.unique():
    for age in percentages.age_start.unique():
        if age < 50 or age > 85:
            continue
            
        plt.figure() 
        no_trt = percentages.loc[(percentages.sex==sex) & (percentages.age_start==age) & (percentages.sbp_medication=='no_treatment')]
        plt.errorbar(no_trt['step'], no_trt[0], yerr=no_trt['final_error'], marker='o')
        #one_half = percentages.loc[(percentages.sex==sex) & (percentages.age_start==age) & (percentages.sbp_medication=='one_drug_half_dose_efficacy')]
        #plt.plot(one_half['step'], one_half[0], marker='o')
        #one_std = percentages.loc[(percentages.sex==sex) & (percentages.age_start==age) & (percentages.sbp_medication=='one_drug_std_dose_efficacy')]
        #plt.plot(one_std['step'], one_std[0], marker='o')
        #two_half = percentages.loc[(percentages.sex==sex) & (percentages.age_start==age) & (percentages.sbp_medication=='two_drug_half_dose_efficacy')]
        #plt.plot(two_half['step'], two_half[0], marker='o')
        #two_std = percentages.loc[(percentages.sex==sex) & (percentages.age_start==age) & (percentages.sbp_medication=='two_drug_std_dose_efficacy')]
        #plt.plot(two_std['step'], two_std[0], marker='o')
        #three_half = percentages.loc[(percentages.sex==sex) & (percentages.age_start==age) & (percentages.sbp_medication=='three_drug_half_dose_efficacy')]
        #plt.plot(three_half['step'], three_half[0], marker='o')
        #three_std = percentages.loc[(percentages.sex==sex) & (percentages.age_start==age) & (percentages.sbp_medication=='three_drug_std_dose_efficacy')]
        #plt.plot(three_std['step'], three_std[0], marker='o')
        plt.title(f'Medications: {sex} {age}') 
        plt.xticks(rotation=90)
        plt.ylabel('Percent')
        plt.xlabel('Time Steps')
        plt.legend(['No Treatment', 'One Drug at Half Dose','One Drug at Std Dose','Two Drug at Half Dose','Two Drug at Std Dose','Three Drug at Half Dose','Three Drug at Std Dose'],loc='center left', bbox_to_anchor=(1, 0.5))
        plt.grid()